In [72]:

from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [73]:
!python -m pip install mlflow

In [74]:

import os
import warnings
import sys
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import xgboost as xgb
from sklearn.pipeline import Pipeline
from sklearn.pipeline import Pipeline


In [75]:
client = mlflow.tracking.MlflowClient()

import logging

logging.basicConfig(level=logging.WARN)
logger = logging.getLogger(__name__)


In [76]:
def eval_metrics(actual, pred):
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2

  
    
 

In [80]:
def build_data(data_path):
   # Read the wine-quality csv file from the URL
    data = pd.read_csv(data_path, low_memory=False,encoding='latin-1', na_values=['(NA)']).fillna(0)
    for column in data.columns:
         data[column] = pd.Categorical(data[column]).codes
      # Split the data into training and test sets. (0.75, 0.25) split.
    train, test = train_test_split(data)
    train_x = train.drop(["Sold_Amount"], axis=1)
    test_x = test.drop(["Sold_Amount"], axis=1)
    train_y = train[["Sold_Amount"]]
    test_y = test[["Sold_Amount"]]
    return train_x, test_x, train_y, test_y 





In [103]:
train_x, test_x, train_y, test_y = build_data(r'/gdrive/My Drive/Colab Notebooks/datium_train.csv')

def xgb_model(training_data, test_data,  max_depth, ntrees, lr):
    
    with mlflow.start_run():
        xgbRegressor = xgb.XGBRegressor(
        max_depth=max_depth,
        n_estimators=ntrees,
        learning_rate=lr,
        random_state=42,
        seed=42,
        subsample=0.75,
        colsample_bytree=0.75,
        reg_lambda=1,
        gamma=1,
        objective = "reg:squarederror"

)
        #pipeline = Pipeline(steps=[("regressor", xgbRegressor)])


        xgbRegressor.fit(train_x, train_y)
        #,early_stopping_rounds=20,eval_set=[(test_x, test_y)])

        predicted_qualities = xgbRegressor.predict(test_x)

        (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)

        print('rmse":',rmse)
        print('r2',r2)
        print('mae',mae)
        mlflow.log_metric("rmse", rmse)
        mlflow.log_metric("r2", r2)
        mlflow.log_metric("mae", mae)

        mlflow.sklearn.log_model(xgbRegressor, "model") 
        feature_importances = pd.DataFrame(xgbRegressor.feature_importances_, index=train_x.columns.tolist(), columns=['importance'])
        feature_importances.sort_values('importance', ascending=False)
  

if __name__ == "__main__":
    xgb_model(train_y,test_y, 3,130,0.3)

rmse": 211.17184
r2 0.8980106649592777
mae 146.9664


In [104]:
#Prediction


In [105]:
train_x, test_x, train_y, test_y = build_data(r'/gdrive/My Drive/Colab Notebooks/datium_test.csv')


In [106]:
xgb_model(train_y,test_y, 3,130,0.3)

rmse": 122.22432
r2 0.9464962281214215
mae 89.237976
